In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from influxdb import *
from influxdb import DataFrameClient
import datetime

In [2]:
INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
TARGET_MEASUREMENT = 'Energy24'

con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = 'select "time","EM4" from ' + TARGET_MEASUREMENT + ' where time > now() - 150d '
df = pd.DataFrame(con_obj.query(query).get_points())
df['time'] = df['time'].astype('datetime64[ns]')
df['time'] = df['time'] + datetime.timedelta(hours=5, minutes=30)
#df = df[df['DeviceID'] == 'EM22']

In [3]:
# start = pd.datetime.now().date() - pd.offsets.Day(2)
# stop = pd.datetime.now().date() - pd.offsets.Day(1)
# df1 = df[(df['time'] >= start) & (df['time'] < stop)]

In [4]:
print(df.time.min())
print(df.time.max())

2020-05-29 20:00:00
2020-09-18 12:00:00


In [5]:
df=df.set_index('time')

In [6]:
len(df)

2651

In [7]:
#df1.to_excel("EM22_08_25_2020.xlsx")

In [8]:
Time_range = pd.DataFrame(pd.date_range(start=(df.index.min()), end=(df.index.max()), freq='H'))
Time_range = Time_range.rename(columns={0: "time"})
Time_range.set_index("time", inplace=True)
df = df.merge(Time_range, how="outer", right_index=True, left_index=True)
df = df.interpolate(method='time')
df = df.fillna(0)

In [9]:
len(Time_range)

2681

In [10]:
len(df)

2681

In [11]:
#df.to_excel("Energy2020-05-29to2020-09-17.xlsx")